In [18]:
### Скелет Нейросети
import numpy
import scipy, scipy.ndimage
import imageio
import os
import matplotlib.pyplot
%matplotlib inline



# Определение класса нейронной сети
class neuralNetwork:
    
    # Инициализируем нейронную сеть
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # Задать количество узлов во входном, скрытом и выходном слое
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        # Коэффициент обучения
        self.lr = learningrate
        
        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # Тренировка нейронной сети
    def train(self, inputs_list, targets_list):
        # Преобразовать список входных значений в двумерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # Рассчитать входиящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # Рассчитать исходиящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # Рассчитать 
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        final_outputs = self.activation_function(final_inputs)
        
        # ошибка = целевое значение - фактическое значение
        output_errors = targets - final_outputs
        
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # Обновить весовые коэффициенты связей между скрытым и выходным слоями
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # Обновить весовые коэффициенты связей между входным и скрытым слоями
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # Опрос нейронной сети
    
    def query(self, inputs_list):
        # преобразовать список входных значений
        # в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot (self.who, hidden_outputs)
        
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [19]:

### создаем нейросеть

# Количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 1000
output_nodes = 5

# Коэффициент обучения равен:
learning_rate = 0.35

#создаем экземпляр нейронной сети
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
# загрузим в список тренировочный набор данных mnist

# кол-во циклов тренировки (эпох)
epochs = 50

# Тренировка Нейронной сети
for e in range(epochs):
    for i in range(5):
        for N in range(10):

            filepath = "Data_training/" + str(i)+"/"+str(i)+str(N)+".png"
            img_array = imageio.imread(filepath)
            img_data = 255.0 - img_array.reshape(784)
            inputs = (img_data / 255.0 * 0.99) + 0.01

            targets = numpy.zeros(output_nodes) + 0.01
            targets[i] = 0.99


            # дополнительные версии изображжения с поворотом +- 10 градусов
            inputs_plusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
            inputs_minusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, order=1, reshape=False)


            n.train(inputs, targets)
            n.train(inputs_plusx_img.reshape(784), targets)
            n.train(inputs_minusx_img.reshape(784), targets)
    
        pass
pass



In [20]:
### проверяем нейросеть

koff = 0 # Счетчик правильных ответов нейросети
Digits = ['А', 'Б','В','Г','Д']

directory = 'Data_test/'
for filename in os.listdir(directory):
    img_array = imageio.imread(directory + filename)
    img_data = 255.0 - img_array.reshape(784)
    img_data = (img_data / 255.0 * 0.99) + 0.01
    outputs = n.query(img_data)
    label = numpy.argmax(outputs)
    

    if label == int(filename[0]):
        koff += 1
    
    print(Digits[int(filename[0])], Digits[label])

print ("эффективность = ", koff / 10)

А А
А В
Б В
Б В
В В
В В
Г Г
Г Г
Д Д
Д Д
эффективность =  0.7
